In [2]:
import mysql.connector as mysql
from mysql.connector import Error
import pandas as pd

In [16]:
dataset = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv")
dataset.head(10)

,index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...
5,5,03/05/1915,01:00,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,NaN,41.0,21.0,0.0,Crashed into trees while attempting to land af...
6,6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,NaN,NaN,Zeppelin L-10 (airship),NaN,NaN,19.0,19.0,0.0,"Exploded and burned near Neuwerk Island, when..."
7,7,07/28/1916,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,20.0,0.0,"Crashed near the Black Sea, cause unknown."
8,8,09/24/1916,01:00,"Billericay, England",Military - German Navy,NaN,NaN,Zeppelin L-32 (airship),NaN,NaN,22.0,22.0,0.0,Shot down by British aircraft crashing in flames.
9,9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,NaN,NaN,Zeppelin L-31 (airship),NaN,NaN,19.0,19.0,0.0,Shot down in flames by the British 39th Home D...


In [18]:
# Selections using the loc method are based on the index of the data frame (if any). Where the index is set on a
# DataFrame, using df.set_index(), the .loc method directly selects based on index values of any rows.
air_name = dataset.set_index('Operator')

military = air_name.loc['Military - U.S. Army']

In [19]:
military.head(20)

,index,Date,Time,Location,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
Operator,,,,,,,,,,,,,
Military - U.S. Army,0,09/17/1908,17:18,"Fort Myer, Virginia",NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
Military - U.S. Army,643,03/22/1944,NaN,New Guinea,NaN,Port Moreaby - Nadzab,Consolidated B-24 Liberator,NaN,NaN,21.0,21.0,NaN,Disappeared while en route on a non-combat mis...
Military - U.S. Army,746,11/11/1945,NaN,Himalayas,NaN,"Chabau, India - Karachi, Pakistan",Douglas C-54,NaN,NaN,44.0,44.0,0.0,Crashed into a mountain ridge 200 ft. short of...
Military - U.S. Army,751,12/08/1945,NaN,"Near Billings, Montana",NaN,"Newark, N.J. - Seattle, WA",Douglas C-47,NaN,NaN,21.0,17.0,0.0,Crashed into a field after hitting trees while...
Military - U.S. Army,1640,01/10/1960,15:38,Gulf of Sirte,NaN,Misurata - Benghazi,de Havilland Canada U-1A Otter,55-2974,47,10.0,10.0,0.0,"Missing, never found."
Military - U.S. Army,2050,05/04/1966,08:00,"Near Di Linh, South Vietnam",NaN,NaN,Boeing CH47A (helicopter),64-13138,NaN,20.0,20.0,0.0,Engine failure caused the helicopter to crash ...
Military - U.S. Army,2075,10/04/1966,NaN,"An Khe, Vietnam",NaN,NaN,de Havilland Canada CV-2B Caribou,63-9751,209,32.0,14.0,0.0,While attempting a go-around the plane hit Hon...
Military - U.S. Army,2229,06/25/1968,07:30,"Near Bearcat, South Vietnam",NaN,NaN,Bell UH-1H / Bell UH-1H / Bell UH-1H,NaN,NaN,29.0,29.0,0.0,A mid-air collision between three helicopters ...
Military - U.S. Army,2303,04/02/1969,12:40,"Near Quang Tri, South Vietnam",NaN,NaN,Boeing Vertol CH47C (helicopter),67-18523,NaN,78.0,24.0,0.0,As the helicopter was taking off in a vertical...


In [17]:
db_name='airplane_crashes'
try:
    mydb = mysql.connect(host='localhost', user='root', password='franci22') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")

        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

[('electronics',), ('GoShare',), ('information_schema',), ('mid_practice',), ('movie',), ('movies',), ('mysql',), ('parents',), ('performance_schema',), ('sys',)]
Database is created


In [ ]:
mycursor.execute("USE" + db_name)